In [98]:
# Cargamos las librerías para tener un entorno de trabajo adecuado
import re
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
stopwords_sp = stopwords.words('spanish')

from nltk.stem.snowball import SnowballStemmer
spanishStemmer=SnowballStemmer("spanish")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [99]:
#Punto 1: preprocesamiento: leeremos el archivo princesas y crearemos una nueva columna con el texto limpio

In [100]:
import pandas as pd

In [101]:
princesas = pd.read_csv("Princesas.csv")
princesas

,Princesa,Personalidad
0,Blancanieves,Blancanieves es una princesa de noble cuna que...
1,Cenicienta,Cenicienta es inicialmente una sirvienta en su...
2,Aurora,"La Princesa Aurora, la Bella Durmiente, es la ..."
3,Bella,Bella es una muchacha que vive en la campiña f...
4,Jasmín,"Cuando se introdujo por primera vez, la Prince..."
5,Pocahontas,"El nombre de Pocahontas significa ""Pequeña Sil..."
6,Mulan,Mulan es atípica a los anteriores papeles feme...
7,Tiana,Es una joven camarera que sueña con ser dueña ...
8,Mérida,Mérida llama la atención por su característico...
9,Moana,"Moana, una joven de 16 años de edad, hija únic..."


In [102]:
from nltk.corpus import stopwords
stopwords_sp = stopwords.words('spanish')

In [103]:
#Aquí hacemos el preprocesamiento de los datos con la función que hemos venido trabajando en clase

def procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = " ".join([palabra for palabra in texto.split() if palabra not in stopwords_sp])
    texto = " ".join([spanishStemmer.stem(palabra) for palabra in texto.split()])
    return texto

In [104]:
princesas['procesado'] = princesas['Personalidad'].apply(lambda texto: procesado(texto))

In [105]:
princesas

,Princesa,Personalidad,procesado
0,Blancanieves,Blancanieves es una princesa de noble cuna que...,blancaniev princes nobl cun ve forz servidumbr...
1,Cenicienta,Cenicienta es inicialmente una sirvienta en su...,cenicient inicial sirvient cas constant objet ...
2,Aurora,"La Princesa Aurora, la Bella Durmiente, es la ...",princes auror bell durmient hij unic rein flor...
3,Bella,Bella es una muchacha que vive en la campiña f...,bell muchach viv campiñ frances padr inventor ...
4,Jasmín,"Cuando se introdujo por primera vez, la Prince...",introduj primer vez princes jasmin poc dias de...
5,Pocahontas,"El nombre de Pocahontas significa ""Pequeña Sil...",nombr pocahont signif pequeñ silenci bas figur...
6,Mulan,Mulan es atípica a los anteriores papeles feme...,mul atip anterior papel femenin pelicul disney...
7,Tiana,Es una joven camarera que sueña con ser dueña ...,jov camarer sueñ ser dueñ propi restaur algun ...
8,Mérida,Mérida llama la atención por su característico...,mer llam atencion caracterist pel anaranj oscu...
9,Moana,"Moana, una joven de 16 años de edad, hija únic...",moan jov años edad hij unic sucesor import jef...


In [106]:
#Punto 2 Crearemos la matriz TF-IDF
#Primero replicaremos la versión larga...

In [107]:
import re
import pandas as pd
import numpy as np

In [108]:
princesas['conteo'] = princesas['procesado'].apply(lambda fila:len(fila.split()))


In [109]:
princesas

,Princesa,Personalidad,procesado,conteo
0,Blancanieves,Blancanieves es una princesa de noble cuna que...,blancaniev princes nobl cun ve forz servidumbr...,118
1,Cenicienta,Cenicienta es inicialmente una sirvienta en su...,cenicient inicial sirvient cas constant objet ...,94
2,Aurora,"La Princesa Aurora, la Bella Durmiente, es la ...",princes auror bell durmient hij unic rein flor...,192
3,Bella,Bella es una muchacha que vive en la campiña f...,bell muchach viv campiñ frances padr inventor ...,125
4,Jasmín,"Cuando se introdujo por primera vez, la Prince...",introduj primer vez princes jasmin poc dias de...,47
5,Pocahontas,"El nombre de Pocahontas significa ""Pequeña Sil...",nombr pocahont signif pequeñ silenci bas figur...,107
6,Mulan,Mulan es atípica a los anteriores papeles feme...,mul atip anterior papel femenin pelicul disney...,71
7,Tiana,Es una joven camarera que sueña con ser dueña ...,jov camarer sueñ ser dueñ propi restaur algun ...,57
8,Mérida,Mérida llama la atención por su característico...,mer llam atencion caracterist pel anaranj oscu...,85
9,Moana,"Moana, una joven de 16 años de edad, hija únic...",moan jov años edad hij unic sucesor import jef...,90


In [110]:

from sklearn.feature_extraction.text import CountVectorizer

In [111]:
count_vect = CountVectorizer()
bow_rep = count_vect.fit_transform(princesas['procesado'].values)

In [128]:
princesas

,Princesa,Personalidad,procesado,conteo
0,Blancanieves,Blancanieves es una princesa de noble cuna que...,blancaniev princes nobl cun ve forz servidumbr...,118
1,Cenicienta,Cenicienta es inicialmente una sirvienta en su...,cenicient inicial sirvient cas constant objet ...,94
2,Aurora,"La Princesa Aurora, la Bella Durmiente, es la ...",princes auror bell durmient hij unic rein flor...,192
3,Bella,Bella es una muchacha que vive en la campiña f...,bell muchach viv campiñ frances padr inventor ...,125
4,Jasmín,"Cuando se introdujo por primera vez, la Prince...",introduj primer vez princes jasmin poc dias de...,47
5,Pocahontas,"El nombre de Pocahontas significa ""Pequeña Sil...",nombr pocahont signif pequeñ silenci bas figur...,107
6,Mulan,Mulan es atípica a los anteriores papeles feme...,mul atip anterior papel femenin pelicul disney...,71
7,Tiana,Es una joven camarera que sueña con ser dueña ...,jov camarer sueñ ser dueñ propi restaur algun ...,57
8,Mérida,Mérida llama la atención por su característico...,mer llam atencion caracterist pel anaranj oscu...,85
9,Moana,"Moana, una joven de 16 años de edad, hija únic...",moan jov años edad hij unic sucesor import jef...,90


In [129]:
tf = pd.DataFrame(bow_rep.toarray())
tf.columns = count_vect.get_feature_names()
tf.index = princesas['Princesa'].index


In [132]:

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect=TfidfVectorizer()
tfidf=tfidf_vect.fit_transform(princesas['procesado'].values)


In [133]:
tfidf.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.06536527, 0.        , 0.05556645, ..., 0.06536527, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.1218657 ,
        0.        ],
       [0.        , 0.        , 0.07952636, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.10212206, 0.        , ..., 0.        , 0.        ,
        0.10212206]])

In [134]:
tfidf_matrix=pd.DataFrame(tfidf.toarray(), columns=tfidf_vect.get_feature_names())
tfidf_matrix.index=princesas.Princesa

In [135]:
tfidf_matrix

,abrum,abuel,acuerd,ademas,afortun,ahor,aisl,ajetre,ajust,alde,...,vid,viej,vigor,vision,vist,viv,volunt,volv,vudu,wak
Princesa,,,,,,,,,,,,,,,,,,,,,
Blancanieves,0.000000,0.000000,0.000000,0.000000,0.086789,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.086789,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
Cenicienta,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.064953,0.000000,0.00000,0.000000,0.000000,0.000000
Aurora,0.065365,0.000000,0.055566,0.000000,0.000000,0.000000,0.065365,0.000000,0.000000,0.000000,...,0.055566,0.000000,0.00000,0.000000,0.043221,0.000000,0.00000,0.065365,0.000000,0.000000
Bella,0.000000,0.000000,0.000000,0.000000,0.000000,0.086927,0.000000,0.000000,0.000000,0.086927,...,0.073896,0.000000,0.00000,0.086927,0.057479,0.073896,0.00000,0.000000,0.000000,0.000000
Jasmín,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.16125,0.000000,0.000000,0.000000,0.16125,0.000000,0.000000,0.000000
Pocahontas,0.000000,0.000000,0.000000,0.099173,0.000000,0.000000,0.000000,0.099173,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
Mulan,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.126019,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
Tiana,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.121866,0.000000
Mérida,0.000000,0.000000,0.079526,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.061858,0.000000,0.00000,0.000000,0.000000,0.000000


In [136]:
# Punto 3...Ahora presentaremos la matriz transpuesta


In [137]:
tfidf_matrix=tfidf_matrix.T.round(3)
tfidf_matrix

Princesa,Blancanieves,Cenicienta,Aurora,Bella,Jasmín,Pocahontas,Mulan,Tiana,Mérida,Moana
abrum,0.000,0.0,0.065,0.000,0.000,0.000,0.0,0.000,0.00,0.000
abuel,0.000,0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.00,0.102
acuerd,0.000,0.0,0.056,0.000,0.000,0.000,0.0,0.000,0.08,0.000
ademas,0.000,0.0,0.000,0.000,0.000,0.099,0.0,0.000,0.00,0.000
afortun,0.087,0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.00,0.000
...,...,...,...,...,...,...,...,...,...,...
viv,0.000,0.0,0.000,0.074,0.000,0.000,0.0,0.000,0.00,0.087
volunt,0.000,0.0,0.000,0.000,0.161,0.000,0.0,0.000,0.00,0.000
volv,0.000,0.0,0.065,0.000,0.000,0.000,0.0,0.000,0.00,0.000
vudu,0.000,0.0,0.000,0.000,0.000,0.000,0.0,0.122,0.00,0.000


In [138]:
#Ahora calcularemos la distancia del coseno

In [139]:
from sklearn.metrics.pairwise import cosine_distances
dist_cos=cosine_distances(tfidf_matrix.T.values)
dist_cos

array([[0.        , 0.84641984, 0.84222342, 0.91811225, 0.93510277,
        0.92216525, 0.88414743, 0.96008221, 0.98513555, 0.95129788],
       [0.84641984, 0.        , 0.85841214, 0.94724288, 0.95920941,
        0.96047919, 0.9326626 , 0.91863944, 0.97083921, 0.97492543],
       [0.84222342, 0.85841214, 0.        , 0.83250296, 0.93719489,
        0.86899415, 0.91949959, 0.93923079, 0.94072328, 0.96593014],
       [0.91811225, 0.94724288, 0.83250296, 0.        , 0.88821874,
        0.89051295, 0.88216931, 0.97333802, 0.94360666, 0.9235464 ],
       [0.93510277, 0.95920941, 0.93719489, 0.88821874, 0.        ,
        0.94275348, 0.93519759, 0.93786763, 0.97250478, 0.96892906],
       [0.92216525, 0.96047919, 0.86899415, 0.89051295, 0.94275348,
        0.        , 0.88884436, 0.96891832, 0.97964557, 0.93609085],
       [0.88414743, 0.9326626 , 0.91949959, 0.88216931, 0.93519759,
        0.88884436, 0.        , 0.99116979, 0.98726676, 0.94584374],
       [0.96008221, 0.91863944, 0.9392307

In [140]:
dist_cos=cosine_distances(tfidf_matrix.T.values)
dist_cos= pd.DataFrame(dist_cos, columns = tfidf_matrix.columns, index = tfidf_matrix.columns)
dist_cos


Princesa,Blancanieves,Cenicienta,Aurora,Bella,Jasmín,Pocahontas,Mulan,Tiana,Mérida,Moana
Princesa,,,,,,,,,,
Blancanieves,0.000000,0.846420,0.842223,0.918112,0.935103,0.922165,0.884147,0.960082,0.985136,0.951298
Cenicienta,0.846420,0.000000,0.858412,0.947243,0.959209,0.960479,0.932663,0.918639,0.970839,0.974925
Aurora,0.842223,0.858412,0.000000,0.832503,0.937195,0.868994,0.919500,0.939231,0.940723,0.965930
Bella,0.918112,0.947243,0.832503,0.000000,0.888219,0.890513,0.882169,0.973338,0.943607,0.923546
Jasmín,0.935103,0.959209,0.937195,0.888219,0.000000,0.942753,0.935198,0.937868,0.972505,0.968929
Pocahontas,0.922165,0.960479,0.868994,0.890513,0.942753,0.000000,0.888844,0.968918,0.979646,0.936091
Mulan,0.884147,0.932663,0.919500,0.882169,0.935198,0.888844,0.000000,0.991170,0.987267,0.945844
Tiana,0.960082,0.918639,0.939231,0.973338,0.937868,0.968918,0.991170,0.000000,0.979275,0.971443
Mérida,0.985136,0.970839,0.940723,0.943607,0.972505,0.979646,0.987267,0.979275,0.000000,0.971818


In [ ]:
#Las princesas más parecidas... aquellas que se acercan a 0
#Lo primero que hay que advertir es que, sorpresivamente, las relaciones no son muy estrechas entre las princesas.
#Sin embargo, podemos concluir que:

#***Blancanieves con Cenicienta y Aurora
#***Cenicienta con Blancanieves y Aurora
#***Aurora con Blancanieves, Cenicienta, Aurora, Bella
#Bella con Aurora
#Jasmin con Bella
#Pocahontas con Aurora y Mulan
#Mulan con Blancanieves, Bella y Pocahontas
#Tiana con Cenicienta (aunque esta relación no es muy significativa)
#Mérida y Moana con ... ella parece ser la creación más auténtica de Disney

#De esta relación podemos concluir que al momento de crear una princesa Diseny, se toman rasgos similares de los personajes anteriores para crearlos.
#Así la categoría princesa tiene algunos elementos en común: descendiente de alguna familia importante, contar con un poder especial,
#Salvar el mundo o tener una aventura en la que la salvan o salva a sus amigos o seres queridos.



In [ ]:
# Las princesas menos parecidas...entendemos que NO son parecidas cuando puntuan >= a 0.95

#Blancanieves con Tiana, Mérida y Moana
#Cenicienta con Jasmin, Pocahontas, Mérida y Moana
#Aurora con Moana
#Bella con Tiana
#Jasmín con Cenicienta, Mérida y Moana
#Pocahontas con Cenicienta, Tiana y Mérida
#Mulan con Mérida
#Tiana con Blancanieves, Bella, Pocahontas, Mulan, Mérida y Moana

#**Mérida Blancanieves, Cenicienta, Jasmín, Pocahontas, Tiana y Moana
#**Moana con Blancanieves, Cenicienta, Aurora, Jasmin, Mulan y Mérida

#De este análisis podemos concluir que no hay una relación entre aquellas princesas con carácter de heroinas o guerreras y 
#aquellas cuya historia se desenvuelve por una aventrua en la que son salvadas o encuentran el amor. 

#**De ahí que una princesa como "Mérida" o "Mulan",  recientes creaciones de Disney, sean opuestas a las princesas clásicas
#como Blancanieves, Cenicienta o Tiana. 

